# Сравнение мер раннего обнаружения изменений в трех временных рядах
В этом ноутбуке для каждого ряда:
1. Загружаем и визуализируем сырые данные
2. Вычисляем коэффициент асимметрии и эксцесса в скользящем окне (30) и наращивающемся (min 30)
3. Визуализируем результаты
После этого сравниваем полученные метрики между рядами.


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')


In [ ]:
def preprocess(df, date_col, value_col):
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col).sort_index()
    df[value_col] = df[value_col].interpolate().ffill().bfill()
    return df

def compute_skew_kurt(series, window):
    skew = series.rolling(window).apply(lambda x: stats.skew(x), raw=False)
    kurt = series.rolling(window).apply(lambda x: stats.kurtosis(x), raw=False)
    exp_skew = series.expanding(min_periods=window).apply(lambda x: stats.skew(x), raw=False)
    exp_kurt = series.expanding(min_periods=window).apply(lambda x: stats.kurtosis(x), raw=False)
    return skew, kurt, exp_skew, exp_kurt


## Ряд: COVID-19 Cases

In [ ]:
# Загрузка и предобработка
df = pd.read_csv(r'/mnt/data/daily-new-confirmed-covid-19-cases-per-million-people.csv')
df_proc = preprocess(df, df.columns[0], df.columns[1])

# Визуализация сырых данных
plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(df.iloc[:,0]), df.iloc[:,1], label='Raw Data')
plt.title('Исходные данные: COVID-19 Cases')
plt.xlabel('Дата')
plt.ylabel(df.columns[1])
plt.legend()
plt.show()

# Вычисление skew и kurtosis
series = df_proc.iloc[:,0]
skew_s, kurt_s, skew_e, kurt_e = compute_skew_kurt(series, 30)

# Визуализация метрик
fig, ax = plt.subplots(1,2, figsize=(12,4))
skew_s.plot(ax=ax[0], label='Rolling Skew (30)')
kurt_s.plot(ax=ax[0], label='Rolling Kurt (30)')
ax[0].set_title('Скользящие метрики')
ax[0].legend()

skew_e.plot(ax=ax[1], label='Exp Skew (min 30)')
kurt_e.plot(ax=ax[1], label='Exp Kurt (min 30)')
ax[1].set_title('Наращивающиеся метрики')
ax[1].legend()
plt.tight_layout()


## Ряд: USD/RUB Exchange Rate

In [ ]:
# Загрузка и предобработка
df = pd.read_csv(r'/mnt/data/Прошлые данные - USD_RUB.csv')
df_proc = preprocess(df, df.columns[0], df.columns[1])

# Визуализация сырых данных
plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(df.iloc[:,0]), df.iloc[:,1], label='Raw Data')
plt.title('Исходные данные: USD/RUB Exchange Rate')
plt.xlabel('Дата')
plt.ylabel(df.columns[1])
plt.legend()
plt.show()

# Вычисление skew и kurtosis
series = df_proc.iloc[:,0]
skew_s, kurt_s, skew_e, kurt_e = compute_skew_kurt(series, 30)

# Визуализация метрик
fig, ax = plt.subplots(1,2, figsize=(12,4))
skew_s.plot(ax=ax[0], label='Rolling Skew (30)')
kurt_s.plot(ax=ax[0], label='Rolling Kurt (30)')
ax[0].set_title('Скользящие метрики')
ax[0].legend()

skew_e.plot(ax=ax[1], label='Exp Skew (min 30)')
kurt_e.plot(ax=ax[1], label='Exp Kurt (min 30)')
ax[1].set_title('Наращивающиеся метрики')
ax[1].legend()
plt.tight_layout()


## Ряд: Chocolate Sales

In [ ]:
# Загрузка и предобработка
df = pd.read_csv(r'/mnt/data/Chocolate Sales.csv')
df_proc = preprocess(df, df.columns[0], df.columns[1])

# Визуализация сырых данных
plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(df.iloc[:,0]), df.iloc[:,1], label='Raw Data')
plt.title('Исходные данные: Chocolate Sales')
plt.xlabel('Дата')
plt.ylabel(df.columns[1])
plt.legend()
plt.show()

# Вычисление skew и kurtosis
series = df_proc.iloc[:,0]
skew_s, kurt_s, skew_e, kurt_e = compute_skew_kurt(series, 30)

# Визуализация метрик
fig, ax = plt.subplots(1,2, figsize=(12,4))
skew_s.plot(ax=ax[0], label='Rolling Skew (30)')
kurt_s.plot(ax=ax[0], label='Rolling Kurt (30)')
ax[0].set_title('Скользящие метрики')
ax[0].legend()

skew_e.plot(ax=ax[1], label='Exp Skew (min 30)')
kurt_e.plot(ax=ax[1], label='Exp Kurt (min 30)')
ax[1].set_title('Наращивающиеся метрики')
ax[1].legend()
plt.tight_layout()


## Сравнение Rolling Skew (window=30) между рядами

In [ ]:
# Собираем rolling skew для каждого ряда
comparison = pd.DataFrame()
for title, path in datasets:
    df = pd.read_csv(path)
    df_proc = preprocess(df, df.columns[0], df.columns[1])
    skew_s, _, _, _ = compute_skew_kurt(df_proc.iloc[:,0], 30)
    comparison[title] = skew_s.values

comparison.index = pd.to_datetime(pd.read_csv(datasets[0][1]).iloc[:,0])
plt.figure(figsize=(10,5))
for col in comparison.columns:
    plt.plot(comparison.index, comparison[col], label=col)
plt.title('Rolling Skew Comparison (30)')
plt.xlabel('Дата')
plt.legend()
plt.show()

## Выводы и рекомендации
- Каждый ряд показал свои особенности: всплески skew/kurt в точках изменений для COVID и USD/RUB.
- Ряд Chocolate Sales почти стабильный, без значительных аномалий по асимметрии.
- Rolling skew шириной 30 точек хорошо выявляет изменения.

**Рекомендации:** применять rolling skew на сырых данных для раннего обнаружения изменений.